In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
)

In [8]:
cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

In [9]:
memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=20,
    return_messages=True,
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

In [10]:
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question)
    print(result)
    memory.save_context({"input": question}, {"output": result.content})

In [11]:
invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")

content="According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they are charged with, despite having a memory that contradicts this belief. He concludes that he had never seen the photograph that disproved their guilt and that it had never existed, indicating that he has accepted the Party's narrative." additional_kwargs={} response_metadata={'token_usage': <OpenAIObject at 0x32f62ec90> JSON: {
  "prompt_tokens": 2323,
  "completion_tokens": 69,
  "total_tokens": 2392,
  "prompt_tokens_details": {
    "cached_tokens": 0,
    "audio_tokens": 0
  },
  "completion_tokens_details": {
    "reasoning_tokens": 0,
    "audio_tokens": 0,
    "accepted_prediction_tokens": 0,
    "rejected_prediction_tokens": 0
  }
}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'stop', 'logprobs': None} id='run-169841ff-119f-4c44-b269-6325d28f769d-0'
content='Winston traced "2+2=5" in the dust on the table.' addi

In [12]:
load_memory({})

[HumanMessage(content='Is Aaronson guilty?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they are charged with, despite having a memory that contradicts this belief. He concludes that he had never seen the photograph that disproved their guilt and that it had never existed, indicating that he has accepted the Party's narrative.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What message did he write in the table?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Winston traced "2+2=5" in the dust on the table.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Who is Julia?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Julia is a character with whom Winston has a romantic relationship. She is someone he loves deeply, and he experiences a strong emotional connection to her, 